In [1]:
import asyncio
import aiohttp
from datetime import datetime, timedelta

In [15]:
async def get_advert_spend(account: str, date_from: str, date_to: str, api_token: str, session: aiohttp.ClientSession):
    """Получает данные по рекламным затратам за указанный период"""
    url = "https://advert-api.wildberries.ru/adv/v1/upd"
    headers = {
        "Authorization": api_token
    }
    params = {
        "from": date_from,
        "to": date_to
        }
    # Интервал для запросов на ВБ
    delay = 1
    # Количество попыток в случае ошибки
    attempts = 3
    for attempt in range(attempts+1):
        try:
                async with session.get(url, headers=headers, params=params, timeout=15) as res:
                    # 1. Пытаемся распарсить JSON, если это возможно
                    content_type = res.headers.get('Content-Type', '')
                    data = await res.json() if 'application/json' in content_type else None

                    if res.status == 200:
                        for d in data:
                            d['account'] = account.upper() # Добавляем информацию об аккаунте в данные
                        print(f"✅ [{account}] Данные получены за {date_from}:{date_to}")
                        return data             

                    # 2. Безопасно достаем описание ошибки
                    error_detail = data.get('detail') if data else await res.text()
                    
                    # 3. Обработка ошибок без дублирования кода
                    if res.status == 401:
                        print(f"🔑 [{account}] Ошибка 401: Неверный токен. ({error_detail})")
                    elif res.status == 429:
                        print(f"⏳ [{account}] Ошибка 429: Лимит запросов! ({error_detail})")
                        asyncio.sleep(delay)
                        continue
                    elif res.status == 400:
                        print(f"❓ [{account}] Ошибка 400: Плохой запрос. ({error_detail})")
                        asyncio.sleep(delay)
                    elif res.status == 503:
                        print(f"❓ [{account}] Ошибка 503: Сервис недоступен. ({error_detail})")
                        asyncio.sleep(delay)
                        continue
                    else:
                        print(f"❌ [{account}] Ошибка {res.status}: {error_detail}")
                    
                    return None
                    
        except aiohttp.ClientError as e:
                print(f"💥 [{account}] Ошибка сессии: {e}")
                return None
        except asyncio.TimeoutError as e:
                print(f"💥 [{account}] Таймаут: {e}")
                return None 
        except Exception as e:
                print(f"💥 [{account}] Непредвиденная ошибка: {e}")
                return None                


async def fetch_advert_spend_info(tokens: dict, date_from = (datetime.now()-timedelta(days=28)).strftime('%Y-%m-%d'), date_to = (datetime.now()-timedelta(days=1)).strftime('%Y-%m-%d')):
    """Асинхронная функция для получения информации о рекламных кампаниях для всех аккаунтов и токенов."""
    # Асинхронно обрабатываем все аккаунты и токены
    async with aiohttp.ClientSession() as session:
            # Создаем задачи для каждого аккаунта и токена
            tasks = [get_advert_spend(account, date_from, date_to, token,session) for account, token in tokens.items()]
            # Ожидаем завершения всех задач и собираем результаты
            results = await asyncio.gather(*tasks)
            return results   

In [16]:
# Импортируем библиотеки для работы с системой и путями
import os
from  pathlib import Path
import sys
# Установка базовой директории и пути к файлу с учетными данными. Используем конструкцию try-except для обработки возможных ошибок при определении пути для notebook.
try:
    BASE_DIR = Path(__file__).resolve().parents[2]
except NameError:
    BASE_DIR = Path.cwd().resolve().parents[1] 

SRC_DIR = os.path.join(BASE_DIR, 'src')

# Добавляем src в пути поиска модулей
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# Импорт самописных утилит
from core.utils_general import load_api_tokens


In [17]:
import pandas as pd




def process_advert_spend_info(data):
    """ Функция обрабатывает полученные данные по рекламным затратам за один день. Метод АПИ возвращает лишние даты, фунцкия так же производит фильтрацию"""
    if data:
        # Список для хранения данных
        data_list = []
        # Проходим внешним циклом по данным каждого ЛК
        for orders_lk in data:
            # Достаем отдельно данные по каждому заказу и добваляем в общий список
            for orders in orders_lk:
                data_list.append(orders)
        df = pd.DataFrame(data_list)
        df['date'] = pd.to_datetime(df['updTime'],
                                      format='ISO8601'
                                      ).dt.date
        return df
    else:
        return pd.DataFrame() 

In [18]:
spend_info = await(fetch_advert_spend_info(load_api_tokens()))
df = process_advert_spend_info(spend_info)

✅ [ВЕКТОР3522] Данные получены за 2026-01-13:2026-02-09
✅ [СТАРТ9237] Данные получены за 2026-01-13:2026-02-09
✅ [ВЕКТОР7794] Данные получены за 2026-01-13:2026-02-09
✅ [СТАРТ8748] Данные получены за 2026-01-13:2026-02-09
✅ [СТАРТ4040] Данные получены за 2026-01-13:2026-02-09
✅ [Вектор2] Данные получены за 2026-01-13:2026-02-09
✅ [ВЕКТОР4216] Данные получены за 2026-01-13:2026-02-09
✅ [Лопатина] Данные получены за 2026-01-13:2026-02-09
✅ [Хачатрян] Данные получены за 2026-01-13:2026-02-09
✅ [Даниелян] Данные получены за 2026-01-13:2026-02-09
✅ [Тоноян] Данные получены за 2026-01-13:2026-02-09
✅ [Старт2] Данные получены за 2026-01-13:2026-02-09
✅ [Оганесян] Данные получены за 2026-01-13:2026-02-09
✅ [Старт] Данные получены за 2026-01-13:2026-02-09
✅ [Вектор] Данные получены за 2026-01-13:2026-02-09
✅ [Пилосян] Данные получены за 2026-01-13:2026-02-09


In [ ]:
df[df['advertId'] == 22846420]

,updTime,campName,paymentType,updNum,updSum,advertId,advertType,advertStatus,account,date
3967,2026-02-09T23:59:59+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,0,668,22846420,9,11,ВЕКТОР,2026-02-09
4118,2026-02-09T12:38:10.130393+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,0,1000,22846420,9,11,ВЕКТОР,2026-02-09
4194,2026-02-09T07:41:38.698795+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,0,569,22846420,9,11,ВЕКТОР,2026-02-09
4548,2026-02-08T23:59:59+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,0,431,22846420,9,11,ВЕКТОР,2026-02-08
4692,2026-02-08T13:24:00.40776+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,283314697,1000,22846420,9,11,ВЕКТОР,2026-02-08
4749,2026-02-08T09:30:05.703396+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,283314697,848,22846420,9,11,ВЕКТОР,2026-02-08
5126,2026-02-07T23:59:59+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,283314697,152,22846420,9,11,ВЕКТОР,2026-02-07
5243,2026-02-07T14:56:37.109974+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,283314697,1000,22846420,9,11,ВЕКТОР,2026-02-07
5305,2026-02-07T10:52:07.64737+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,283314697,1000,22846420,9,11,ВЕКТОР,2026-02-07
5379,2026-02-07T01:27:36.897851+03:00,189954586 / Весы напольные / Ильдар / АРК,Баланс,283314697,114,22846420,9,11,ВЕКТОР,2026-02-07


In [ ]:
from sqlalchemy import BigInteger, String, Boolean, DateTime
from core.utils_sql import get_db_engine, sync_data_to_postgres
from modules.WB.advert.schemas import advert_spend_info_dict

def advert_spend():
    """Получение данных по затратам"""
    spend_info = asyncio.run(fetch_advert_spend_info(load_api_tokens()))
    df = process_advert_spend_info(spend_info)
    # Присваиваем имена колонок в snake_case
    df = df.rename(columns={
        'updTime': 'upd_time',
        'campName': 'camp_name',
        'paymentType': 'payment_type',
        'updNum': 'upd_num',
        'updSum': 'upd_sum',
        'advertId': 'advert_id',
        'advertType': 'advert_type',
        'advertStatus': 'advert_status'
                            })
    # Преобразуем датафрейм в словарь для вставкив БД
    data = df.to_dict(orient='records')
    # Установка соединения с базой данных
    engine = get_db_engine()

    table_name = list(advert_spend_info_dict.keys())[0]
    unique_keys = ['advert_id', 'upd_num', 'upd_time']  # Уникальность по ID кампании
    # В ключе указываем имя таблицы в БД
    schema_definition = advert_spend_info_dict.get(table_name)

    sync_data_to_postgres(engine, table_name, data, schema_definition, unique_keys)